# AB Testing - E-Commerce

Hello everyone, in this small project we are going to look at some practical use of A/B Testing. Recently, a company has developed a new web page in order to try and increase the number of users who "convert," meaning the number of users who decide to pay for the company's product. Our goal is to help the company understand if they should implement this new page, keep the old page, or perhaps run the experiment longer to make their decision.

To solve this task, we will define functions that estimate the relevant information out of the samples, compute the relevant statistic given each case and make a decision on whether to (or not) reject the null hypothesis.

IMPORTANT!! The actual metric being tested is "converted" (whether a user made a purchase), which is a binary outcome (0 or 1). The code treats this as if it were a continuous variable. For binary outcomes, it's more common to use a chi-square test or logistic regression rather than a t-test. That said, with large sample sizes (which you have here), the t-test can still provide valid results due to the Central Limit Theorem. The means calculated here represent the proportion of conversions in each group.

Let's get started!

In [1]:
import math
import numpy as np
import pandas as pd
from scipy import stats

The metric we are going to evaluate is `converted`, which means whether the user decided to pay for the company's product.

We are going to declare two groups:
- `control`: These users will keep seeing your original website.


- `treatment`: These users will see your website with the new web page.

In [21]:
# Load the data
data = pd.read_csv("ab_data.csv")

data.head(10)

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
5,936923,20:49.1,control,old_page,0
6,679687,26:46.9,treatment,new_page,1
7,719014,48:29.5,control,old_page,0
8,817355,58:09.0,treatment,new_page,1
9,839785,11:06.6,treatment,new_page,1


In [22]:
print(f"The dataset size is: {len(data)}")

The dataset size is: 294480


In [23]:
# Separate the data from the two groups
control_converted_data = data[data["group"] == "control"]['converted']
treatment_converted_data = data[data["group"] == "treatment"]['converted']

print(f"Control group size: {len(control_converted_data)}")
print(f"Treatment group size: {len(treatment_converted_data)}")
print(f"Control group conversion rate: {control_converted_data.mean()}")
print(f"Treatment group conversion rate: {treatment_converted_data.mean()}")

Control group size: 147202
Treatment group size: 147278
Control group conversion rate: 0.12039917935897611
Treatment group conversion rate: 0.11891796466546259


Notice that the split is not perfectly balanced. This is common in AB testing as there is randomness associated with the way the users are assigned to each group. 

At first glance it looks like the new web page did in fact drive users to stay longer on your website. However you know better than driving conclusions at face value out of this data so you decide to perform a hypothesis test to know if there is a significant difference between the **means** of these two segments. 

<a name="1"></a>
## 1 - The Underlying Mathematics

Remember that your job is to measure if changing the web page leads to an increase of purchases. Rewriting this as hypothesis test, the **null hypothesis** is that the change did not affect the amount of purchases. Let's name the variables:

- $\mu_c$ is the average time a user **in the control group** spend in the website. Recall that the **control group** is the group accessing the website without the change in the background color.
- $\mu_v$ is the average time a user **in the variation groups** spend in the website. Recall that the **variation group** is the groups accessing the website **with the updated background color**.

Also, recall that your intention is to measure if the background color leads to an **increase** in the time a visitor spend in the website. So writing this experiment as a hypothesis test, the **null hypothesis** is then $H_0: \mu_c = \mu_v$ and the **alternative hypothesis** is $H_1: \mu_v > \mu_c$, or equivalently, $H_1: \mu_v - \mu_c > 0$. 

Therefore, the hypothesis you will test is:

$$H_0: \mu_v = \mu_c \quad \text{vs.} \quad H_1: \mu_v - \mu_c > 0$$

As you saw in the lectures, this is a **right-tailed** test, as you are looking for an increase in the average time. As you saw above, you have more than 2000 users per group, this is a great amount of data so it is reasonable to rely in the Central Limit Theorem that the **average time** for each group follows a normal distribution. Remember that this result is for the group **average time** altogether and not that the time each user spend follows a normal distribution. You don't know the exact distribution for the amount of time a user spend in your website, however, the CLT assures that if we gather enough data, their average time will be very close to a normal distribution whose mean is the average time a user spend in the website. Let's then define two new quantities:

- $\overline{X}_c$ - the control group **sample mean**.
- $\overline{X}_v$ - the variation group **sample mean**.
- $n_c$ - the control group **size**.
- $n_v$ - the variation group **size**.

So, by the Central Limit Theorem, you may suppose that

- $$\overline{X}_c \sim N\left(\mu_c, \left(\frac{\sigma_c}{\sqrt{n_c}}\right)^2\right)$$
- $$\overline{X}_v \sim N\left(\mu_v, \left(\frac{\sigma_v}{\sqrt{n_v}}\right)^2\right)$$

Note that with our assumptions of normality, $\overline{X}_v - \overline{X}_c$ also follows a normal distribution. So, if $H_0$ is true, then $\mu_c = \mu_v$ and $\mu_v - \mu_c = 0$, therefore:

$$\overline{X}_c - \overline{X}_v \sim N\left(\mu_v - \mu_c, \left(\dfrac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\dfrac{\sigma_c}{\sqrt{n_c}}\right)^2\right) = N\left(0, \left(\dfrac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\dfrac{\sigma_c}{\sqrt{n_c}}\right)^2\right)$$

Or, equivalently:

$$\frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\left(\frac{\sigma_v}{\sqrt{n_v}}\right)^2 + \left(\frac{\sigma_c}{\sqrt{n_c}}\right)^2}} \sim N(0, 1)$$

However, remember that **you don't know the exact values for** $\sigma_v$ and $\sigma_c$, as they are the **population standard deviation** and you are working with a sample, so the best you can do is compute the **sample standard deviation**. So you must replace $\sigma_c$ and $\sigma_v$ by the sample standard deviation, respectively, $s_c$ and $s_v$. You also saw in the lectures that replacing the population standard deviation by the sample standard deviation changes the random variable from a Normal to a t-student:

$$t = \frac{\left( \overline{X}_v - \overline{X}_c \right)}{\sqrt{\left(\frac{s_v}{\sqrt{n_v}}\right)^2 + \left(\frac{s_c}{\sqrt{n_c}}\right)^2}} \sim t_d$$

Where $d$ is the **degrees of freedom** for this scenario. If we suppose that both groups have the same standard deviation, then $d = n_c + n_v - 2$, however there is no argument supporting this supposition, so the formula for the degrees of freedom gets a bit messier:

$$d = \frac{\left[\frac{s_{v}^2}{n_v} + \frac{s_{c}^2}{n_c} \right]^2}{\frac{(s_{v}^2/n_v)^2}{n_v-1} + \frac{(s_{c}^2/n_c)^2}{n_c-1}}$$

Once you get the actual value for $t_d$ the, with a given significance level $\alpha$, you can decide if this value falls within the range of values that are likely to occur in the $t$-student distribution (where 'likely' is related with your significance level). To perform this step you must find the value $p$ such that 

$$p = P(t_d > t | H_0)$$

If this value is less than your significance level $\alpha$, then you **reject the null hypothesis**, because it means that you observed a value that is very unlikely to occur (unlikely here means that is less than the significance level you have set) if $H_0$ is true.

Also, remember that $P(t_d \leq t)$ is the $\text{CDF}$ (cumulative distribution function) for the $t$-student distribution with $d$ degrees of freedom in the point $x = t$, so to compute $P(t_d > t)$ you may compute:

$$P(t_d > t) = 1 - \text{CDF}_{t_d}(t)$$

Since $P(t_d \leq t) + P(t_d > t) = 1$

<a name="4"></a>
## 4 - Step by step computation


Wrapping up everything discussed above:

The hypothesis test is given by:

$$H_0: \mu_v = \mu_c \quad \text{vs.} \quad H_1: \mu_v - \mu_c > 0$$

You will start computing:

- $n_c$ and $n_v$, the control and variation group sizes, respectively.
- $\overline{X}_c$ and $\overline{X}_v$, the average time spent by the users in the control and variation group, respectively. 
- $s_c$ and $s_v$, the **sample** standard deviation for the time spend by the users in the control and variation group, respectively.

With these quantities in hand, the next steps are to compute:

- $d$, the degrees of freedom of the $t$-student distribution, $t_d$.
- The $t$-value, which it will be called $t$.
- The $p$ value for the distribution $t_d$ for the $t$-value, i.e., the value  $p = P(t_d > t | H_0)$.

Finally, for a given significance level $\alpha$, you will be able to decide if you reject or not $H_0$, depending on wether $p \leq \alpha$ or not.

Let's get your hands into work now! Run the cell below to retrieve the session times for the control and variation groups.

In [16]:
# X_c stores the session tome for the control group and X_t, for the treatment group. 
X_c = control_converted_data.to_numpy()
X_t = treatment_converted_data.to_numpy()   

In [18]:
print(f"The first 10 entries for X_c are:\n{X_c[:20]}\n")
print(f"The first 10 entries for X_v are:\n{X_t[:20]}\n")

The first 10 entries for X_c are:
[0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0]

The first 10 entries for X_v are:
[0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1]



In [19]:
# Calculate the basic statistics for the control group
def get_stats(X):
    """
    Calculate basic statistics of a given data set.

    Parameters:
    X (numpy.array): Input data.

    Returns:
    tuple: A tuple containing:
        - n (int): Number of elements in the data set.
        - x (float): Mean of the data set.
        - s (float): Sample standard deviation of the data set.
    """
    
    # Get the group size
    n = len(X)
    # Get the group mean
    x = np.mean(X)
    # Get the group sample standard deviation (do not forget to pass the parameter ddof if using the method .std)
    s = np.std(X, ddof=1)

    return (n,x,s)

In [43]:
n_c, x_c, s_c = get_stats(X_c)
n_t, x_t, s_t = get_stats(X_t)

print(f"Control group size: {n_c}, mean: {x_c:.3f}, standard deviation: {s_c:.3f}")
print(f"Treatment group size: {n_t}, mean: {x_t:.3f}, standard deviation: {s_t:.3f}")

Control group size: 147202, mean: 0.120, standard deviation: 0.325
Treatment group size: 147278, mean: 0.119, standard deviation: 0.324


Next, let's compute the degrees of freedom

$$
\nu \approx \frac{\left(\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}\right)^2}{\frac{(s_1^2/n_1)^2}{n_1 - 1} + \frac{(s_2^2/n_2)^2}{n_2 - 1}}
$$
Where:

$\nu$ is the degrees of freedom
$s_1^2$ and $s_2^2$ are the sample variances
$n_1$ and $n_2$ are the sample size

In [28]:
# Compute the degrees of freedom
def degrees_of_freedom(n_t, s_t, n_c, s_c):
    """Computes the degrees of freedom for two samples.

    Args:
        control_metrics (estimation_metrics_cont): The metrics for the control sample.
        treatment_metrics (estimation_metrics_cont): The metrics for the treatment sample.

    Returns:
        numpy.float: The degrees of freedom.
    """

    s_t_n_t = s_t**2 / n_t

    # Compute s_c^2/n_c (remember to use Python syntax or np.square)
    s_c_n_c = s_c**2 / n_c

    # Compute the numerator in the formula given above
    numerator = (s_t_n_t + s_c_n_c)**2

    # Also, remember to use parenthesis to indicate the operation order. Note that a/b+1 is different from a/(b+1).
    denominator = (s_c_n_c**2) / (n_c-1) + (s_t_n_t**2) / (n_t-1)
    
    ### END CODE HERE ###

    dof = numerator/denominator
        
    return dof

In [44]:
d = degrees_of_freedom(n_t, s_t, n_c, s_c)
print(f"The degrees of freedom for the t-student in this scenario is: {d:.2f}")

The degrees of freedom for the t-student in this scenario is: 294467.87


<a name="ex03"></a>
### T-value

In this exercise, you will compute the $t$-value, given by

$$t = \frac{\left( \overline{X}_t - \overline{X}_c \right)}{\sqrt{\left(\frac{s_t}{\sqrt{n_t}}\right)^2 + \left(\frac{s_c}{\sqrt{n_c}}\right)^2}} = \frac{\left( \overline{X}_t - \overline{X}_c \right)}{\sqrt{\frac{s_t^2}{n_t} + \frac{s_c^2}{n_c}}}$$

Remember that you are storing $\overline{X}_c$ and $\overline{X}_t$ in the variables $x_c$ and $x_t$, respectively. 

In [31]:
def t_value(n_t, x_t, s_t, n_c, x_c, s_c):

    # Compute s_t^2/n_t 
    s_t_n_t = np.square(s_t) / n_t

    # Compute s_c^2/n_c (remember to use Python syntax or np.square)
    s_c_n_c = np.square(s_c) / n_c

    # Compute the numerator for the t-value as given in the formula above
    numerator = x_t - x_c

    # Compute the denominator for the t-value as given in the formula above
    denominator = np.sqrt(s_t_n_t + s_c_n_c)

    t = numerator/denominator

    return t

In [45]:
t = t_value(n_t, x_t, s_t, n_c, x_c, s_c)
print(f"The t-value for this experiment is: {t:.3f}")

The t-value for this experiment is: -1.238


<a name="ex04"></a>
### P_value

In this exercise, you will compute the $p$ value for $t_d$, for a given significance level $\alpha$. Recall that this experiment is a right-tailed t-test, because you are investigating whether the new web page increases the amount of purchases overall. 

In this experiment the $p$-value for a significance level of $\alpha$ is given by

$$p = P(t_d > t) = 1 - \text{CDF}_{t_d}(t)$$

Example:

Suppose you want to compute the CDF for a $t$-student distribution with $d = 10$ degrees of freedom for a t-value of $1.21$.

In [36]:
t_10 = stats.t(df = 10)
cdf = t_10.cdf(1.21)
print(f"The CDF for the t-student distribution with 10 degrees of freedom and t-value = 1.21, or equivalently P(t_10 < 1.21) is equal to: {cdf:.2f}")

The CDF for the t-student distribution with 10 degrees of freedom and t-value = 1.21, or equivalently P(t_10 < 1.21) is equal to: 0.87


In [37]:
# Compute the p-value
def p_value(d, t_value):

    # Load the t-student distribution with $d$ degrees of freedom. Remember that the parameter in the stats.t is given by df.
    t_d = stats.t(df=d)

    # Compute the p-value, P(t_d > t). Remember to use the t_d.cdf with the proper adjustments as discussed above.
    p = 1 - t_d.cdf(t_value)

    return p

<a name="ex05"></a>
### Wrapping up

In this exercise you will wrap up all the functions you have built so far to decide if you accept $H_0$ or not, given a significance level of $\alpha$.

It will input both control and treatment groups and it will output `Reject H_0$` or `Do not reject H_0` accordingly.

Remember that you **reject** $H_0$ if the p-value is **less than** $\alpha$. 

In [40]:
def make_decision(X_t, X_c, alpha = 0.05):

    # Compute n_v, x_t and s_t
    n_t, x_t, s_t = get_stats(X_t)

    # Compute n_c, x_c and s_c
    n_c, x_c, s_c = get_stats(X_c)

    # Compute the degrees of freedom for the t-student distribution for this experiment.
    # Also, remember that x_c and x_v are not used in this computation
    d = degrees_of_freedom(n_t, s_t, n_c, s_c)
    
    # Compute the t-value
    t = t_value(n_t, x_t, s_t, n_c, x_c, s_c)

    # Compute the p-value for the t-student distribution with d degrees of freedom
    p = p_value(d, t)

    # This is the decision step. Compare p with alpha to decide about rejecting H_0 or not. 
    if p < alpha:
        return 'Reject H_0'
    else:
        return 'Do not reject H_0'


In [47]:
alphas = [0.06, 0.05, 0.04, 0.01]
for alpha in alphas:
    print(f"For an alpha of {alpha} the decision is to: {make_decision(X_t, X_c, alpha = alpha)}")

For an alpha of 0.06 the decision is to: Do not reject H_0
For an alpha of 0.05 the decision is to: Do not reject H_0
For an alpha of 0.04 the decision is to: Do not reject H_0
For an alpha of 0.01 the decision is to: Do not reject H_0
